In [ ]:
!pip install -q pandas sentence-transformers faiss-cpu transformers accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 52.1 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving airline_faq.csv to airline_faq.csv


In [ ]:
!pip install faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 75.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv("airline_faq.csv")
df.head()        # show first few rows
df.columns       # list all column names


Index(['Question', 'Answer'], dtype='object')

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

# Load the dataset
df = pd.read_csv("airline_faq.csv")

# ✅ Use correct column names
questions = df["Question"].tolist()
answers = df["Answer"].tolist()

# Create embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
question_embeddings = model.encode(questions)

# Create FAISS index
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(question_embeddings))

# Save the index and metadata
faiss.write_index(index, "faq_index.faiss")
with open("faq_metadata.pkl", "wb") as f:
    pickle.dump({"questions": questions, "answers": answers}, f)

print("✅ Index and metadata saved successfully!")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Error: No runtime connected'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Index and metadata saved successfully!


In [ ]:
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# Load the FAISS index and metadata
index = faiss.read_index("faq_index.faiss")
with open("faq_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

questions = metadata["questions"]
answers = metadata["answers"]

# Load the same embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

def retrieve_faq(query, top_k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    print("🔍 Top retrieved FAQs:\n")
    for i, idx in enumerate(indices[0]):
        print(f"{i+1}. {questions[idx]}")
        print(f"   → {answers[idx]}\n")

# Example query
retrieve_faq("What is Aurora Skies baggage policy?")


🔍 Top retrieved FAQs:

1. How can I modify my Aurora Skies Airways booking?
   → You can access your booking online to modify your flight, rebook on a different flight, or explore other options. Change and cancellation fees will be determined by the fare rules associated with your ticket.

2. How does Aurora Skies Airways handle schedule changes for travel agents?
   → Aurora Skies Airways has established guidelines for travel agents to address the re-protection of clients affected by schedule changes. These guidelines cover various scenarios, including flight cancellations, time changes, and routing or airport changes.

3. What happens if Aurora Skies Airways changes my flight schedule?
   → If Aurora Skies Airways changes your flight schedule resulting in a departure or arrival time change of 3 hours or more, you are entitled to a refund.



In [ ]:
!pip install -q transformers accelerate torch


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# RAG-style pipeline
def rag_chatbot(query, top_k=3):
    # Retrieve top relevant FAQs
    query_embedding = model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    retrieved_context = ""
    for idx in indices[0]:
        retrieved_context += f"Q: {questions[idx]}\nA: {answers[idx]}\n\n"

    # Build the prompt
    prompt = f"""You are an AI assistant for Aurora Skies Airways.
Use only the following FAQ context to answer the question accurately.

Context:
{retrieved_context}

Question: {query}

Answer:"""

    # Generate grounded answer
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = gen_model.generate(**inputs, max_new_tokens=150)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# 🔹 Example query
user_query = "Can I get a refund if my flight is cancelled?"
print("💬 User:", user_query)
print("🤖 Chatbot:", rag_chatbot(user_query))


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

💬 User: Can I get a refund if my flight is cancelled?
🤖 Chatbot: Yes, Aurora Skies Airways allows refunds within 24 hours of purchase for all fare types, including published and net fares, as well as tickets with codeshare and interline flights. This policy does not apply to group fares or fares purchased for same-day travel.


In [ ]:
print("✈️ Aurora Skies Airways Chatbot")
print("Type 'exit' to stop chatting.\n")

while True:
    user_query = input("💬 You: ")
    if user_query.lower() == "exit":
        print("👋 Goodbye! Have a nice flight!")
        break
    response = rag_chatbot(user_query)
    print(f"🤖 Bot: {response}\n")


✈️ Aurora Skies Airways Chatbot
Type 'exit' to stop chatting.

💬 You: How Much Baggage I can carry
🤖 Bot: You can carry up to a maximum of 2,000 pounds.

💬 You: how are you tho
🤖 Bot: not enough information

💬 You: what is the name of your flight
🤖 Bot: Aurora Skies Airways flight

💬 You: can i jump from the plane
🤖 Bot: No, you cannot jump from the plane.

💬 You: can i hijack the plane
🤖 Bot: No, Aurora Skies Airways does not allow passengers to hijack a plane.

💬 You: how much people can i take with my single ticker
🤖 Bot: One ticker can take up to a maximum of ten people.

💬 You: exit
👋 Goodbye! Have a nice flight!


In [ ]:
!pip install streamlit


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 135.4 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load saved data
index = faiss.read_index("faq_index.faiss")
with open("faq_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

questions = metadata["questions"]
answers = metadata["answers"]

# Load models
embedder = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Define chatbot function
def rag_chatbot(query, top_k=3):
    query_embedding = embedder.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)

    context = ""
    for idx in indices[0]:
        context += f"Q: {questions[idx]}\nA: {answers[idx]}\n\n"

    prompt = f"""You are an assistant for Aurora Skies Airways.
Use only the FAQ context to answer accurately.

Context:
{context}

Question: {query}
Answer:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    outputs = gen_model.generate(**inputs, max_new_tokens=150)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Streamlit UI
st.set_page_config(page_title="Aurora Skies Airways Chatbot", page_icon="✈️")
st.title("✈️ Aurora Skies Airways Chatbot")

user_query = st.text_input("Ask your question:")
if st.button("Get Answer"):
    if user_query.strip():
        with st.spinner("Generating answer..."):
            response = rag_chatbot(user_query)
        st.success(response)
    else:
        st.warning("Please enter a question.")


Writing app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


⠙

⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹your url is: https://flat-breads-poke.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.207.176:8501

2025-10-30 18:24:30.911146: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761848670.933663   11929 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761848670.940598   11929 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1761848670.958423   11929 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1761848670.9584

In [ ]:
!curl https://loca.lt/mytunnelpassword

34.125.207.176